In [ ]:
import numpy as np

# ファイルパス
file_paths = [
    "DeBERTa-large-L-898-with-probs.txt",
    "DeBERTa-large-H-912-with-probs.txt",
    "robertaH-with-probs.txt",
    "robertaL-with-probs.txt"
]

# 各ファイルの重み
weights = [1, 1, 1, 0]

# ラベルの総数
num_labels = 6

# 結果ファイルを開く
with open('result.txt', 'w') as result_file:
    # 各ファイルを処理
    for line_idx in range(sum(1 for _ in open(file_paths[0]))):  # 最初のファイルの行数で繰り返し
        label_probs = np.zeros(num_labels)

        # 各ファイルを順に処理
        for file_path, weight in zip(file_paths, weights):
            with open(file_path, 'r') as f:
                # 行をスキップして現在の行に進む
                for i, line in enumerate(f):
                    if i == line_idx:
                        # 行をパース
                        parts = line.strip().split(", ")
                        label = int(parts[0])  # 一番左のラベル
                        probs = [float(prob.split(": ")[1]) for prob in parts[1:]]  # 確率の部分

                        # 重み付きで確率を加算
                        label_probs += np.array(probs) * weight
                        break

        # 最も高い確率を持つラベルを選択
        max_label = np.argmax(label_probs) + 1

        # 結果をファイルに書き出し
        result_file.write(f"{max_label}\n")

In [ ]:
import numpy as np

# ファイルパス
file_paths = [
    "DeBERTa-large-L-898-with-probs.txt",
    "DeBERTa-large-H-912-with-probs.txt",
    "robertaH-with-probs.txt",
    "robertaL-with-probs.txt"
]

# 各ファイルの重み
weights = [1, 1, 1, 0]

# ラベルの総数
num_labels = 6

# 結果ファイルを開く
with open('result.txt', 'w') as result_file:
    # 各ファイルを処理
    for line_idx in range(sum(1 for _ in open(file_paths[0]))):  # 最初のファイルの行数で繰り返し
        label_probs = np.zeros(num_labels)

        # 各ファイルを順に処理
        for file_path, weight in zip(file_paths, weights):
            with open(file_path, 'r') as f:
                # 行をスキップして現在の行に進む
                for i, line in enumerate(f):
                    if i == line_idx:
                        # 行をパース
                        parts = line.strip().split(", ")
                        label = int(parts[0])  # 一番左のラベル
                        probs = [float(prob.split(": ")[1]) for prob in parts[1:]]  # 確率の部分

                        # 重み付きで確率を加算
                        label_probs += np.array(probs) * weight
                        break

        # 最も高い確率を持つラベルを選択
        max_label = np.argmax(label_probs) + 1

        # 結果をファイルに書き出し
        result_file.write(f"{max_label}\n")

In [ ]:
import numpy as np
from sklearn.metrics import cohen_kappa_score

def load_gold_labels(filepath):
    labels_H, labels_L = [], []
    with open(filepath) as f:
        for line in f:
            array = line.strip().split('\t')

            labels_H.append(max(int(array[0]), int(array[1])))
            labels_L.append(min(int(array[0]), int(array[1])))

    return np.array(labels_H), np.array(labels_L)

def get_gold_labels(predictions, lower_labels, higher_labels):
    if np.sum(predictions == lower_labels) >= np.sum(predictions == higher_labels):
        gold_labels = lower_labels
        gold_labels[predictions == higher_labels] = higher_labels[predictions == higher_labels]
    else:
        gold_labels = higher_labels
        gold_labels[predictions == lower_labels] = lower_labels[predictions == lower_labels]
    return gold_labels

filepath = "dev.label" # 正解ラベルのパス
predict_file = "result.txt" # 予測ラベルのパス

predictions = []
with open(predict_file, "r") as f:
    for line in f:
        predictions.append(int(line.strip()))

gold_H, gold_L = load_gold_labels(filepath)
gold = get_gold_labels(predictions, gold_H, gold_L)
kappa = cohen_kappa_score(predictions, gold, weights='quadratic') # QWKの計測
print(kappa)

0.9322206162567915
